## _Computer Vision_
### Lab 5, _Image Interpolation_.
#### >> Topics  :
* ##### Interpolation Methods
* ##### Inpainting
* ##### Image Pyramids
* ##### Edges vs Contours
* ##### Hough Space
* ##### Hough Circles Detector

##### >> Note  :
> ##### to close image windows smoothly please **press Esc** on your keyboard, **don't close** it directly by clicking on 'X' to avoid kernel interruption.
> - ##### To run this lab using google colab follow below instructions:
> 1. Import `cv2_imshow` method using `from google.colab.patches import cv2_imshow`.
> 2. Replace all `cv.imshow()` with `cv2_imshow()` with one parameter **name of that image**.
> 3. Specify your path.

In [32]:
import cv2 as cv
import numpy as np

### Section 0, _Helper Functions_:
> ##### This cell contains some helper function such as error_handler, please run it without any modification.

In [33]:
# please don't modify this code.
def show_text_window(titles):
    black = np.zeros((len(titles) * 150, 400))
    for idx, t in enumerate(titles):
        place = idx + 1
        cv.putText(black, t, (10, place * 100),
                   cv.FONT_HERSHEY_SIMPLEX, 0.75, (200, 0, 200), 1, 2)
    cv.imshow("Values", black)


def get_updated_value(key, value, **kwargs):
    if key == ord('+'):
        return value, 0, 0
    elif key == ord('-'):
        return -value, 0, 0
    elif key == ord('*'):
        return 0, kwargs.get('value2', value), 0
    elif key == ord('/'):
        return 0, -kwargs.get('value2', value), 0
    elif key == ord('8'):
        return 0, 0, kwargs.get('value3', value)
    elif key == ord('2'):
        return 0, 0, -kwargs.get('value3', value)
    elif key == 27:
        raise Exception('Terminated by user!')
    else:
        return 0, 0, 0


def error_handler(func):
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except Exception as ex:
            cv.destroyAllWindows()
            print(f'{ex}')

    return wrapper

### Section 1, _Interpolation Methods_:
* ##### Interpolation methods are used to estimate pixel values at non-integer coordinates when resizing images.
* ##### The choice of interpolation method affects the quality of the resized image.
* ##### [Click to check the documentation](https://docs.opencv.org/3.4/da/d54/group__imgproc__transform.html)
![image info](https://upload.wikimedia.org/wikipedia/commons/thumb/9/90/Comparison_of_1D_and_2D_interpolation.svg/768px-Comparison_of_1D_and_2D_interpolation.svg.png)

In [34]:
@error_handler
def apply_resize_with_interpolation(path):
    img = cv.imread(path)
    cv.imshow("Image", img)
    dimensions = img.shape

    interpolation_types = [cv.INTER_NEAREST, cv.INTER_LINEAR, cv.INTER_CUBIC]
    interpolation_names = [
        "Nearest Interpolation", "Linear Interpolation", "Cubic Interpolation",
    ]

    scale_ratio = 1
    while True:
        for interpolation_type, interpolation_name in zip(
            interpolation_types, interpolation_names
        ):
            # resize with interpolation
            interpolated = cv.resize(
                img,
                (int(dimensions[1] * scale_ratio), int(dimensions[0] * scale_ratio)),
                interpolation=interpolation_type,
            )
            cv.imshow(interpolation_name, interpolated)

        show_text_window([f"Scale: {scale_ratio:.1f}"])

        k = cv.waitKey(0)
        ds, _, _ = get_updated_value(k, 0.1)
        scale_ratio += ds
        scale_ratio = max(scale_ratio, 0.1)


path = "../Images/Lenna.png"
apply_resize_with_interpolation(path)

Terminated by user!


In [35]:
path = "../Images/nature.jpg"
apply_resize_with_interpolation(path)

Terminated by user!


### Section 2, _Inpainting_:
* ##### It's Used in image processing and computer vision to fill in-missing or damaged parts of an image with plausible content.
* ##### The goal of inpainting is to restore or complete an image by estimating the missing information based on the surrounding image context
* ##### Several algorithms were designed for this purpose and OpenCV provides two of them.
* ##### cv.INPAINT_TELEA Algorithm: starts from the boundary of the region and goes inside the region gradually filling everything in the boundary first. It takes a small neighbourhood around the pixel on the neighbourhood to be inpainted. This pixel is replaced by normalized weighted sum of all the known pixels in the neighbourhood.


* ##### [Click to check the documentation](https://docs.opencv.org/3.4/df/d3d/tutorial_py_inpainting.html)

In [36]:
@error_handler
def inpainting(img_path, mask_path):
    image = cv.imread(img_path)
    mask = cv.imread(mask_path, 0)

    # make sure that the dimensions are equal.
    assert image.shape[:2] == mask.shape[:2]

    # Radius of a circular neighborhood of each point inpainted that is considered by the algorithm.
    inpaint_radius = 1
    while True:
        inpainted_img = cv.inpaint(image, mask, inpaint_radius, cv.INPAINT_TELEA)

        cv.imshow("OriginalImage", image)
        cv.imshow('mask', mask)
        cv.imshow('Result', inpainted_img)

        show_text_window([f'Radius: {inpaint_radius}'])

        k = cv.waitKey(0)
        dr, _, _ = get_updated_value(k, 1)
        inpaint_radius += dr


img_path = "../Images/messi.jpg"
mask_path = "../Images/mask.png"

inpainting(img_path, mask_path)

Terminated by user!


In [37]:
img_path = "../Images/messi 2.png"
mask_path = "../Images/mask 2.png"

inpainting(img_path, mask_path)

Terminated by user!


### Section 3, _Image Pyramids_:
* ##### They are a multi-scale representation of an image at different resolutions.
* ##### They are constructed by repeatedly applying a smoothing and down-sampling operation to the original image to generate a series of images at different scales.

* ##### cv.pyrDown: it performs the downsampling step of the Gaussian pyramid construction. First, it convolves the source image with 5x5 Gaussian kernel and then it downsamples the image by rejecting even rows and columns.


* ##### cv.pyrUp: it performs the upsampling step of the Gaussian pyramid construction. First, it upsamples the source image by injecting even zero rows and columns and then convolves the result with the same kernel as in pyrDown multiplied by 4.

* ##### [Click to check the documentation](https://docs.opencv.org/3.4/d4/d1f/tutorial_pyramids.html)
![image info](https://upload.wikimedia.org/wikipedia/commons/thumb/4/43/Image_pyramid.svg/800px-Image_pyramid.svg.png)

In [38]:
@error_handler
def pyramid(path):
    img = cv.imread(path)
    lower_res = cv.pyrDown(img)
    upper_res = cv.pyrUp(img)
    show_text_window([f'Image Dimensions: {img.shape[:2]}', f'Lower Dimensions: {lower_res.shape[:2]}',
                      f'Upper Dimensions: {upper_res.shape[:2]}'])

    cv.imshow('Original Image', img)
    cv.imshow('Lower Resolution', lower_res)
    cv.imshow('Upper Resolution', upper_res)

    cv.waitKey(0)
    cv.destroyAllWindows()


path = "../Images/home.jpg"
pyramid(path)

### Section 4, _Edges vs Contours_:
* ##### Contours are used to identify and represent the boundaries or outlines of objects in an image.
* ##### They provide information about the shapes and spatial relationships of objects within the image.
* ##### The output of contour detection is a set of curves that represent the boundaries of objects or regions in the image.
* ##### [Click to check the documentation](https://docs.opencv.org/3.4/d4/d73/tutorial_py_contours_begin.html)

In [50]:
@error_handler
def contours(path):
    image = cv.imread(path)
    image = cv.resize(image, (image.shape[1] // 2, image.shape[0] // 2))
    gray = cv.cvtColor(image, cv.COLOR_RGB2GRAY)

    thresh = 100
    while True:
        image_copy = image.copy()
        _, binary = cv.threshold(gray, thresh, 255, cv.THRESH_BINARY)
        cv.imshow("Gray Image", gray)
        cv.imshow("Binary Image", binary)

        contours, _ = cv.findContours(binary, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
        image_copy = cv.drawContours(image_copy, contours, -1, (0, 255, 0), 2)

        cv.imshow("Input Image", image)
        cv.imshow("Contour Detected Image", image_copy)

        show_text_window([f"Threshold: {thresh}"])

        k = cv.waitKey(0)
        dt, _, _ = get_updated_value(k, 10)
        thresh += dt


path = "../Images/sudoku.jpg"
contours(path)

Terminated by user!


In [49]:
path = "../Images/bird.png"
contours(path)

Terminated by user!


### Section 5, _Hough Space_:
* ##### It's a mathematical concept to detect and represent geometric shapes, patterns, and features in an image.
* ##### Hough Transform, a technique originally developed for line detection but later extended to detect other shapes such as circles and ellipses.
* ##### Hough space plays a crucial role in finding and representing these shapes in a parametric form.
* ##### [Click to check the documentation for hough lines.](https://docs.opencv.org/3.4/d9/db0/tutorial_hough_lines.html)
![image info](https://miro.medium.com/v2/resize:fit:1400/0*KoAmeuoTu9Uz0eNW.png)
* ##### [Click to check the documentation for hough circles.](https://docs.opencv.org/3.4/d4/d70/tutorial_hough_circle.html)
![image info](https://www.cis.rit.edu/class/simg782.old/talkHough/circles1.gif)

In [53]:
@error_handler
def circles(path):
    image = cv.imread(path)
    image = cv.medianBlur(image, 5)
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

    minRadius, maxRadius, minDist = 0, 0, 20 
    while True:
        image_copy = image.copy()
        circles = cv.HoughCircles(gray, cv.HOUGH_GRADIENT, dp=1, minDist=minDist, param1=100, param2=25, minRadius=minRadius, maxRadius=maxRadius)
        
        if circles is not None:
            circles = np.uint16(np.around(circles))

            for circle in circles[0]:
                *center, radius = circle
                cv.circle(image_copy, center, radius, (0, 255, 0), 2)
                cv.circle(image_copy, center, 2, (0, 0, 255), 3)

        cv.imshow('Detected Circles', image_copy)

        show_text_window([
            f"Min Radius: {minRadius}", f"Max Radius: {maxRadius}", f"Min Distance: {minDist}",
            ])

        k = cv.waitKey(0)
        dmn, dmx, dds = get_updated_value(k, 10, value3=1)
        minRadius += dmn
        maxRadius += dmx
        minDist += dds

path = "../Images/coins.jpg"
circles(path)

Terminated by user!


## _The End_.